In [1]:

import argparse
import torch
# import lib
import numpy as np
import os
import datetime
from dataset import *
from loss import *
from model import *
from optimizer import *
from trainer import *
from torch.utils import data
import pickle
import sys
from dataset_time_cut import *
from logger import *

import sys
sys.path.insert(0, '../PyTorch_GBW_LM')
sys.path.insert(0, '../PyTorch_GBW_LM/log_uniform')

from sparse_model import RNNModel, SampledSoftmax

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--hidden_size', default=50, type=int)
parser.add_argument('--num_layers', default=1, type=int)
parser.add_argument('--batch_size', default=100, type=int)
parser.add_argument('--dropout_input', default=0, type=float)
parser.add_argument('--dropout_hidden', default=.2, type=float)

# parse the optimizer arguments
parser.add_argument('--optimizer_type', default='Adagrad', type=str)
parser.add_argument('--final_act', default='tanh', type=str)
parser.add_argument('--lr', default=.05, type=float)
parser.add_argument('--weight_decay', default=0.0, type=float)
parser.add_argument('--momentum', default=0.1, type=float)
parser.add_argument('--eps', default=1e-6, type=float)

parser.add_argument("-seed", type=int, default=7,
					 help="Seed for random initialization")
parser.add_argument("-sigma", type=float, default=None,
					 help="init weight -1: range [-sigma, sigma], -2: range [0, sigma]")
parser.add_argument("--embedding_dim", type=int, default=-1,
					 help="using embedding")
# parse the loss type
parser.add_argument('--loss_type', default='TOP1', type=str)
# parser.add_argument('--loss_type', default='BPR', type=str)
parser.add_argument('--topk', default=5, type=int)
# etc
parser.add_argument('--bptt', default=1, type=int)
parser.add_argument('--test_observed', default=5, type=int)
parser.add_argument('--window_size', default=30, type=int)
parser.add_argument('--warm_start', default=5, type=int)

parser.add_argument('--n_epochs', default=20, type=int)
parser.add_argument('--time_sort', default=False, type=bool)
parser.add_argument('--model_name', default='GRU4REC', type=str)
parser.add_argument('--save_dir', default='models', type=str)
parser.add_argument('--data_folder', default='../Data/movielen/1m/', type=str)
parser.add_argument('--data_action', default='item.pickle', type=str)
parser.add_argument('--data_cate', default='cate.pickle', type=str)
parser.add_argument('--data_time', default='time.pickle', type=str)
parser.add_argument("--is_eval", action='store_true')
parser.add_argument('--load_model', default=None,  type=str)
parser.add_argument('--checkpoint_dir', type=str, default='checkpoint')
parser.add_argument('--data_name', default=None, type=str)
parser.add_argument('--shared_embedding', default=None, type=int)
parser.add_argument('--patience', default=1000)
parser.add_argument('--negative_num', default=1000, type=int)

# Get the arguments
args = parser.parse_args(args=[])
args.cuda = torch.cuda.is_available()


In [3]:
args.data_folder = "../Data/tmall/100k_unknown_cate/"
args.data_action = "item_time.pickle"
args.data_cate = "cate_time.pickle"
args.data_time = "time_time.pickle"
args.data_name = "taobao"
args.valid_start_time = 1512172800
valid_start_time = args.valid_start_time
args.test_start_time = 1512259200
test_start_time = args.test_start_time

In [4]:
args.embedding_dim = 300
args.hidden_size = 300
args.lr = 0.0001
args.window_size = 5
args.test_observed = 5
args.n_epochs = 300
args.shared_embedding = 1
args.batch_size = 300
args.optimizer_type = "Adam"

In [12]:
# data_file = "data.pickle"
# f = open(data_file, "rb")
# data_map = pickle.load(f)
# data_obj = data_map["data"]

In [6]:
hidden_size = args.hidden_size
num_layers = args.num_layers
batch_size = args.batch_size
dropout_input = args.dropout_input
dropout_hidden = args.dropout_hidden
embedding_dim = args.embedding_dim
final_act = args.final_act
loss_type = args.loss_type
topk = args.topk
optimizer_type = args.optimizer_type
lr = args.lr
weight_decay = args.weight_decay
momentum = args.momentum
eps = args.eps
BPTT = args.bptt

n_epochs = args.n_epochs
time_sort = args.time_sort

window_size = args.window_size
shared_embedding = args.shared_embedding

if embedding_dim == -1:
    print("embedding dim not -1", embedding_dim)
    raise AssertionError()

observed_threshold = args.test_observed

data_action = args.data_folder+args.data_action
data_cate = args.data_folder+args.data_cate
data_time = args.data_folder+args.data_time

data_obj = Data(data_action, data_cate, data_time, valid_start_time, test_start_time, observed_threshold, window_size)

train_data = data_obj.train_dataset

print("+"*10)
print("valid load")

valid_data = data_obj.test_dataset
test_data = data_obj.test_dataset

input_size = len(data_obj.items)
output_size = input_size

negative_num = args.negative_num
# print("input_size", input_size)

train_data_loader = dataset.DataLoader(train_data, batch_size)

valid_data_loader = dataset.DataLoader(valid_data, batch_size)

data_name = args.data_name



action seq num 51275
cate seq num 51275
time seq num 51275
loading item map
finish loading item map
observed_threshold 5 5
loading data
valid_start_time 1512172800
test start time 1512259200
subseq num for training 2738883
subseq len num for training 2738883
seq idx num for training 2738883
subseq num for testing 430797
subseq len num for testing 430797
++++++++++
valid load
item num 68008


In [7]:
input_size

68008

In [ ]:
index = 20

In [ ]:
file_name = self.m_folder+"/"+str(index)+".pickle"
print("file name", file_name)
f_step = open(file_name, "rb")

step_data_map = pickle.load(f_step)
f_step.close()

x_long_cate_action_list_step = step_data_map["long_action_cate"]
x_long_cateNum_list_step = step_data_map["long_cateNum"]

x_long_cate_actionNum_list_step = step_data_map["long_actionNum_cate"]
x_long_cate_list_step = step_data_map["long_cate"]
x_short_action_list_step = step_data_map["short_action"]
x_short_cate_list_step = step_data_map["short_cate"]
x_short_actionNum_list_step = step_data_map["short_actionNum"]
y_action_step = step_data_map["target_action"]
y_cate_step = step_data_map["target_cate"]
y_action_idx_step = step_data_map["target_id"]

In [12]:
ss = SampledSoftmax(output_size, negative_num, embedding_dim, None)

log = Logger()
log.addIOWriter(args)

model = GRU4REC(log, ss, window_size, input_size, hidden_size, output_size,
                    final_act=final_act,
                    num_layers=num_layers,
                    use_cuda=args.cuda,
                    batch_size=batch_size,
                    dropout_input=dropout_input,
                    dropout_hidden=dropout_hidden,
                    embedding_dim=embedding_dim, 
                    shared_embedding=shared_embedding
                    )

output_file hcdmg2_01_57_19_07_300_300_300_Adam_0.0001_5_1_taobao
share embedding


In [24]:
save_path = "../log/samplePaddingSessionRNN/checkpoint/07190203/model_best.pt"

save_model_data = torch.load(save_path)
# print(save_model_data)
model.load_state_dict(save_model_data['model']) 
model.eval()
recall = save_model_data['recall']
print("recall", recall)

mrr = save_model_data['mrr']
print("mrr", mrr)

recall 0.2988362369337979
mrr 0.09398002545333074


In [15]:
model.look_up.weight.data

tensor([[ 0.0046,  0.4635,  0.2425,  ..., -0.2431, -0.2885,  0.4390],
        [-0.1967,  0.2312, -0.4256,  ...,  0.1649,  0.2620,  0.2347],
        [-0.0548, -0.7392,  0.0220,  ..., -0.6229,  0.6161,  0.9687],
        ...,
        [-0.7595,  0.1734, -0.5826,  ...,  2.1363,  0.0233,  0.0448],
        [-0.9475, -0.9853, -0.1102,  ..., -0.1099, -0.3941,  0.4392],
        [-0.2130,  0.2578,  0.7439,  ...,  0.8787, -0.5507, -0.4998]],
       device='cuda:0')

In [16]:
model.m_ss.params.weight.data

tensor([[ 0.0046,  0.4635,  0.2425,  ..., -0.2431, -0.2885,  0.4390],
        [-0.1967,  0.2312, -0.4256,  ...,  0.1649,  0.2620,  0.2347],
        [-0.0548, -0.7392,  0.0220,  ..., -0.6229,  0.6161,  0.9687],
        ...,
        [-0.7595,  0.1734, -0.5826,  ...,  2.1363,  0.0233,  0.0448],
        [-0.9475, -0.9853, -0.1102,  ..., -0.1099, -0.3941,  0.4392],
        [-0.2130,  0.2578,  0.7439,  ...,  0.8787, -0.5507, -0.4998]],
       device='cuda:0')

recall 0.1413566701497736
mrr 0.08586457241696374


In [32]:
save_path = "../log/samplePaddingSessionRNN/checkpoint/07181627/model_best.pt"

save_model_data = torch.load(save_path)
# print(save_model_data)
model.load_state_dict(save_model_data['model']) 
model.eval()
recall = save_model_data['recall']
print("recall", recall)

mrr = save_model_data['mrr']
print("mrr", mrr)

recall 0.29677351916376304
mrr 0.08824897100078104


In [25]:
loss_type = "XE"
loss_func = LossFunction(loss_type=loss_type, use_cuda=args.cuda)

In [33]:
model.eval()
dataloader = valid_data_loader

losses = []
recalls = []
mrrs = []
weights = []

topk = 5
eval_iter = 0
device = torch.device('cuda')
warm_start = args.warm_start
with torch.no_grad():
    total_test_num = []

    for x_batch, y_batch, x_len_batch, idx_batch in dataloader:        
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        warm_start_mask = (idx_batch>=warm_start)

        hidden = model.init_hidden()

        output_batch = model(x_batch, hidden, x_len_batch)

        # et_1 = datetime.datetime.now()
        # print("duration 1", et_1-st)

        sampled_logit_batch, sampled_target_batch = model.m_ss(output_batch, y_batch)

        loss_batch = loss_func(sampled_logit_batch, sampled_target_batch)
        losses.append(loss_batch.item())

        # et_2 = datetime.datetime.now()
        # print("duration 2", et_2-et_1)

        # logit_batch = self.model.m_ss.params(output_batch)
        recall_batch, mrr_batch = evaluate(sampled_logit_batch, sampled_target_batch, warm_start_mask, k=topk)

        weights.append( int( warm_start_mask.int().sum() ) )
        recalls.append(recall_batch)
        mrrs.append(mrr_batch)

        total_test_num.append(y_batch.view(-1).size(0))

mean_losses = np.mean(losses)
mean_recall = np.average(recalls, weights=weights)
mean_mrr = np.average(mrrs, weights=weights)
print("topk", topk)
print("total_test_num", np.sum(total_test_num))
print("recall", mean_recall)
print("mrr", mean_mrr)

shuffling
batch num 1435
topk 5
total_test_num 430500
recall 0.12722648083623694
mrr 0.07883690043972344


In [13]:
model.eval()
dataloader = valid_data_loader

losses = []
recalls = []
mrrs = []
weights = []

nonzero_cate_num_total = 0

topk = 5
eval_iter = 0
device = torch.device('cuda')
warm_start = args.warm_start
with torch.no_grad():
    total_test_num = []

    for x_cate_batch, mask_cate, max_acticonNum_cate, seqLen_cate, x_batch, mask_batch, seqLen_batch, y_batch, idx_batch in dataloader:        
        x_cate_batch = x_cate_batch.to(device)
        mask_cate = mask_cate.to(device)
        
        x_batch = x_batch.to(device)
        mask_batch = mask_batch.to(device)

        y_batch = y_batch.to(device)
        warm_start_mask = (idx_batch>=warm_start).to(device)
        
        nonzero_cate_num = np.count_nonzero(seqLen_cate)
        nonzero_cate_num_total += nonzero_cate_num
#         print("zero_cate_num", zero_cate_num)
#         print("seqLen_cate", seqLen_cate)
#         print(seqLen_cate.nonzero())

#         exit()
print("nonzero_cate_num_total", nonzero_cate_num_total)


batch_num 4158
zero_cate_num_total 634331


In [34]:
634331.0/831600

0.7627837902837903

In [4]:
np.count_nonzero(np.eye(4))

4

In [40]:
index = 20
folder = "../Data/tmall/100k_unknown_cate/train"
file_name = folder+"/"+str(index)+".pickle"
print("file name", file_name)
f_step = open(file_name, "rb")

step_data_map = pickle.load(f_step)
f_step.close()

x_long_cate_action_list_step = step_data_map["long_action_cate"]
x_long_cateNum_list_step = step_data_map["long_cateNum"]

x_long_cate_actionNum_list_step = step_data_map["long_actionNum_cate"]
x_long_cate_list_step = step_data_map["long_cate"]
x_short_action_list_step = step_data_map["short_action"]
x_short_cate_list_step = step_data_map["short_cate"]
x_short_actionNum_list_step = step_data_map["short_actionNum"]
y_action_step = step_data_map["target_action"]
y_cate_step = step_data_map["target_cate"]
y_action_idx_step = step_data_map["target_id"]

file name ../Data/tmall/100k_unknown_cate/train/20.pickle


In [41]:
len(y_cate_step)

456

In [43]:
len(y_cate_step[0])

300

In [52]:
a = torch.randn((3, 4))

In [59]:
print(a)

tensor([[ 2.6493,  0.8490, -0.3502,  1.2809],
        [ 0.6891,  0.3602, -1.4018,  0.1154],
        [-0.3195,  0.4709, -0.4335,  0.7348]])


In [67]:
b = torch.LongTensor([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

In [68]:
print(b)

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])


In [95]:
c = torch.LongTensor([[2], [4], [12]])
d = b == c

In [96]:
d

tensor([[0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1]], dtype=torch.uint8)

In [97]:
d.sum(dim=1)

tensor([1, 0, 1])

In [98]:
a[d]


tensor([0.8490, 0.7348])

In [99]:
e = torch.zeros(3)
print(e)

tensor([0., 0., 0.])


In [100]:
d.sum(dim=1).nonzero().reshape(-1)

tensor([0, 2])

In [101]:
e[d.sum(dim=1).nonzero().reshape(-1)] = a[d]

In [102]:
e

tensor([0.8490, 0.0000, 0.7348])

In [85]:
d.nonzero()

tensor([[0, 1],
        [1, 2]])

In [54]:
a[d]

tensor([ 0.8490, -1.4018,  0.7348])

In [56]:
d

tensor([[0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]], dtype=torch.uint8)

In [104]:
a = [[1, 2, 3, 4], [1, 3, 4], [20, 30, 40]]
max(max(i) for i in a)

40

In [105]:
a = np.array([1, 2, 3])

In [106]:
b = a-2
print(b)

[-1  0  1]


In [ ]:
max()